Anova_tfidf_150features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/tfidf_withstopwordchangelevels_150_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
1,1,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
2,2,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
3,3,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
4,4,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,1.881556,-0.086085,-0.086085,-0.086085


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head() 

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,-0.333031,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
1,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,-0.333031,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
2,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,-0.333031,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
3,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,-0.333031,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
4,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.113663,-0.048688,-0.256451,-0.070533,1.920714,...,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,1.881556,-0.086085,-0.086085,-0.086085


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3334701644950077


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.339344,0.339344,0.635745,0.254383,0.295802,0.295802,0.400714,0.197242


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9829    0.9557    0.9691       361
           2     0.9962    0.9367    0.9655       553
           3     0.8711    0.9935    0.9282       306

    accuracy                         0.9566      1220
   macro avg     0.9500    0.9620    0.9543      1220
weighted avg     0.9609    0.9566    0.9572      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3652    0.2675    0.3088       157
           2     0.4800    0.6582    0.5552       237
           3     0.2381    0.1538    0.1869       130

    accuracy                         0.4160       524
   macro avg     0.3611    0.3599    0.3503       524
weighted avg     0.3856    0.4160    0.3900       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.339344,0.339344,0.635745,0.254383,0.295802,0.295802,0.400714,0.197242
1,RandomForest,0.956557,0.956557,0.960858,0.957229,0.416031,0.416031,0.385594,0.389995


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5592    0.2355    0.3314       361
           2     0.4990    0.8879    0.6389       553
           3     0.5714    0.1569    0.2462       306

    accuracy                         0.5115      1220
   macro avg     0.5432    0.4267    0.4055      1220
weighted avg     0.5350    0.5115    0.4494      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2817    0.1274    0.1754       157
           2     0.4431    0.7890    0.5675       237
           3     0.1935    0.0462    0.0745       130

    accuracy                         0.4065       524
   macro avg     0.3061    0.3209    0.2725       524
weighted avg     0.3328    0.4065    0.3277       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.469672,0.469672,0.690439,0.319324,0.444656,0.444656,0.278171,0.285503
1,RandomForest,0.961475,0.961475,0.963288,0.961475,0.402672,0.402672,0.368943,0.372587
2,Adaboost,0.511475,0.511475,0.534976,0.449400,0.406489,0.406489,0.332840,0.327743


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9692    0.3490    0.5132       361
           2     0.5550    0.9946    0.7124       553
           3     0.9293    0.3007    0.4543       306

    accuracy                         0.6295      1220
   macro avg     0.8178    0.5481    0.5600      1220
weighted avg     0.7714    0.6295    0.5888      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3971    0.1720    0.2400       157
           2     0.4585    0.8397    0.5931       237
           3     0.2273    0.0385    0.0658       130

    accuracy                         0.4408       524
   macro avg     0.3610    0.3500    0.2996       524
weighted avg     0.3827    0.4408    0.3565       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.469672,0.469672,0.690439,0.319324,0.444656,0.444656,0.278171,0.285503
1,RandomForest,0.961475,0.961475,0.963288,0.961475,0.402672,0.402672,0.368943,0.372587
2,Adaboost,0.511475,0.511475,0.534976,0.449400,0.406489,0.406489,0.332840,0.327743
3,GBM,0.629508,0.629508,0.771449,0.588752,0.440840,0.440840,0.382737,0.356504


**SVM Model**

In [26]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [27]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [28]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4302    0.4183    0.4242       361
           2     0.5670    0.3978    0.4676       553
           3     0.3472    0.5458    0.4244       306

    accuracy                         0.4410      1220
   macro avg     0.4481    0.4540    0.4387      1220
weighted avg     0.4714    0.4410    0.4439      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3481    0.4013    0.3728       157
           2     0.4494    0.2996    0.3595       237
           3     0.2757    0.3923    0.3238       130

    accuracy                         0.3531       524
   macro avg     0.3577    0.3644    0.3520       524
weighted avg     0.3759    0.3531    0.3546       524



In [30]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.339344,0.339344,0.635745,0.254383,0.295802,0.295802,0.400714,0.197242
1,RandomForest,0.956557,0.956557,0.960858,0.957229,0.416031,0.416031,0.385594,0.389995
2,SVM,0.440984,0.440984,0.471393,0.443903,0.353053,0.353053,0.375924,0.354622


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4946    0.5069    0.5007       361
           2     0.5902    0.6980    0.6396       553
           3     0.4133    0.2647    0.3227       306

    accuracy                         0.5328      1220
   macro avg     0.4994    0.4899    0.4877      1220
weighted avg     0.5175    0.5328    0.5190      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3735    0.3949    0.3839       157
           2     0.4843    0.5865    0.5305       237
           3     0.3099    0.1692    0.2189       130

    accuracy                         0.4256       524
   macro avg     0.3892    0.3835    0.3778       524
weighted avg     0.4078    0.4256    0.4093       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.469672,0.469672,0.690439,0.319324,0.444656,0.444656,0.278171,0.285503
1,RandomForest,0.961475,0.961475,0.963288,0.961475,0.402672,0.402672,0.368943,0.372587
2,Adaboost,0.511475,0.511475,0.534976,0.449400,0.406489,0.406489,0.332840,0.327743
3,GBM,0.629508,0.629508,0.771449,0.588752,0.440840,0.440840,0.382737,0.356504
4,SVM,0.484426,0.484426,0.537465,0.375211,0.465649,0.465649,0.432120,0.362735
5,KNN,0.532787,0.532787,0.517538,0.519013,0.425573,0.425573,0.407832,0.409288


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5517    0.0443    0.0821       361
           2     0.5289    0.2152    0.3059       553
           3     0.2671    0.8431    0.4057       306

    accuracy                         0.3221      1220
   macro avg     0.4492    0.3675    0.2645      1220
weighted avg     0.4700    0.3221    0.2647      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3913    0.0573    0.1000       157
           2     0.4790    0.2405    0.3202       237
           3     0.2513    0.7385    0.3750       130

    accuracy                         0.3092       524
   macro avg     0.3739    0.3454    0.2651       524
weighted avg     0.3962    0.3092    0.2678       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.469672,0.469672,0.690439,0.319324,0.444656,0.444656,0.278171,0.285503
1,RandomForest,0.961475,0.961475,0.963288,0.961475,0.402672,0.402672,0.368943,0.372587
2,Adaboost,0.511475,0.511475,0.534976,0.449400,0.406489,0.406489,0.332840,0.327743
3,GBM,0.629508,0.629508,0.771449,0.588752,0.440840,0.440840,0.382737,0.356504
4,SVM,0.484426,0.484426,0.537465,0.375211,0.465649,0.465649,0.432120,0.362735
5,KNN,0.532787,0.532787,0.517538,0.519013,0.425573,0.425573,0.407832,0.409288
6,GNB,0.322131,0.322131,0.469979,0.264690,0.309160,0.309160,0.396233,0.267831
